In [1]:
%cd /Users/santiago/thesis/code

# reload magic
%load_ext autoreload
%autoreload 2

from src import utils
from src.dtw import dtw
from src import jit_dtw

# DATASET = 'Beef' # seq length 470
DATASET = 'ItalyPowerDemand' # seq length 24
# DATASET = 'LargeKitchenAppliances' # seq length 720
S = utils.get_class_sequences(2, DATASET)
a = S[0]
b = S[1]

/Users/santiago/thesis/code


In [69]:
%time _ = dtw(a, b)

CPU times: user 3.14 ms, sys: 32 µs, total: 3.17 ms
Wall time: 3.17 ms


In [85]:
%time c1, _ = jit_dtw.dtw(a, b)
print(c1)

%time c2 = jit_dtw.compute_dtw_cost(a, b)
print(c2)

CPU times: user 137 µs, sys: 1 µs, total: 138 µs
Wall time: 140 µs
CPU times: user 16 µs, sys: 1 µs, total: 17 µs
Wall time: 19.1 µs
